# Geekbench benchmark on Android

Geekbench4 is an app offering several benchmarks to run on android smartphones. The one used in this notebook is the '**CPU**' benchmark, which runs several workloads that follow the lines of what is commonly run by smartphones (AES, JPEG codec, FFT, and so on). The benchmark runs all the tests in '**Single-Core**' mode as well as in '**Multi-Core**' in order to compare the single-thread and multi-thread performances of the device.

**Do note that the benchmark will attempt to upload its results, which includes some hardware information**

In [1]:
from conf import LisaLogging
LisaLogging.setup()

2017-03-22 10:46:08,023 INFO    : root         : Using LISA logging configuration:
2017-03-22 10:46:08,129 INFO    : root         :   /home/vagrant/lisa/logging.conf


In [2]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Support Functions

This function helps us run our experiments:

In [3]:
def experiment():
    
    # Configure governor
    target.cpufreq.set_all_governors('sched')
    
    # Get workload
    wload = Workload.getInstance(te, 'Geekbench')
    
    # Run Geekbench workload
    wload.run(te.res_dir, test_name='CPU', collect='ftrace')
        
    # Dump platform descriptor
    te.platform_dump(te.res_dir)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [4]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'android',
    "board"        : 'pixel',
    
    # Device
    "device"       : "HT67M0300128",
    
    # Android home
    "ANDROID_HOME" : "/home/vagrant/lisa/tools/android-sdk-linux/",

    # Folder where all the results will be collected
    "results_dir" : "Geekbench_example",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_overutilized",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "cpu_capacity",
            "cpu_frequency",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset'],
}

In [ ]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False)
target = te.target

## Workloads execution

This is done using the **experiment** helper function defined above which is configured to run a **Geekbench - CPU** experiment.

In [ ]:
# Initialize Workloads for this test environment
results = experiment()

## Results collection

Geekbench4 uses a baseline score of 4000, which is the benchmark score of an Intel Core i7-6600U. Higher scores are better, with double the score indicating double the performance. You can have a look at the results for several android phones here https://browser.primatelabs.com/android-benchmarks

In [9]:
class TE():
    res_dir="/home/vagrant/lisa/ipynb/scratchpad/geekbench4/sched"
    
te = TE()

In [14]:
def get_geekbench(filepath):
    res = {}
    with open(filepath) as fd:
        res = json.loads(fd.read()) 
    return res

def get_global_results(geekbench):
    data = []
    idx = []
    for section in geekbench["sections"]:
        data.append(section["score"])
        idx.append(section["name"])
    df = pd.DataFrame(data=data, index=idx, columns=["Global score"])
    return df.T

def get_workloads_results(geekbench):    
    data = []
    idx = []
    benchmark_fields = ["score", "threads", "runtime_mean", "rate_string"]

    for section in geekbench["sections"]:
        for workload in section["workloads"]:
            wl_data = []
            for field in benchmark_fields:
                wl_data.append(workload[field])            
            data.append(tuple(wl_data))
            idx.append( (section["name"], workload["name"]) )  

    sections = [section["name"] for section in geekbench["sections"]]
    workloads = set([workload["name"] for section in geekbench["sections"] for workload in section["workloads"]]) 
    idx = pd.MultiIndex.from_tuples(idx)
        
    df = pd.DataFrame(data, index=idx, columns=benchmark_fields).unstack(0)
    df.columns = df.columns.swaplevel(0,1)
    df.sortlevel(0, axis = 1, inplace=True)
    return df

In [19]:
for f in os.listdir(te.res_dir):
    if f.endswith(".gb4"):
        geekbench = get_geekbench(te.res_dir + "/" + f)
        gdf = get_global_results(geekbench)
        wdf = get_workloads_results(geekbench)

        print "---- Global benchmark results ----"
        if geekbench["browser"]["url"] is not None:
            print "Uploaded results available here : " + geekbench["browser"]["url"]
        display(gdf)
        print "---- Detailled benchmark results ----"
        display(wdf)

---- Global benchmark results ----
Uploaded results available here : https://browser.geekbench.com/v4/cpu/2147368


,Single-Core,Multi-Core
Global score,1702,3977


---- Detailled benchmark results ----


Multi-Core                                             \
                            score threads runtime_mean           rate_string   
AES                          2035       4     0.332845           1.53 GB/sec   
Camera                       7115       4     0.217685       19.7 images/sec   
Canny                        6722       4     0.223805      93.2 Mpixels/sec   
Dijkstra                     4906       4     0.613225          3.32 MTE/sec   
Face Detection               5347       4     0.196573  1.56 Msubwindows/sec   
Gaussian Blur                5730       4     0.209446     100.4 Mpixels/sec   
HDR                          7351       4     0.688534      26.7 Mpixels/sec   
HTML5 DOM                    1419       4     1.783401    1.29 MElements/sec   
HTML5 Parse                  4109       4     0.274679           18.7 MB/sec   
Histogram Equalization       4607       4     0.349193     144.0 Mpixels/sec   
JPEG                         7193       4     0.366393      57.9 Mpixels/sec   
LLVM                         4184       4     1.857255   287.7 functions/sec   
LZMA                         4287       4     0.536466           6.70 MB/sec   
Lua                          3473       4     0.369708           3.57 MB/sec   
Memory Bandwidth             3368       2     0.139533           18.0 GB/sec   
Memory Copy                  3806       2     0.381772           10.5 GB/sec   
Memory Latency               2835       2     0.307260  6.55 Moperations/sec   
N-Body Physics               3689       4     0.414314       2.76 Mpairs/sec   
PDF Rendering                5501       4     0.728765     146.2 Mpixels/sec   
Ray Tracing                  4825       4     0.535315     704.7 Kpixels/sec   
Rigid Body Physics           4820       4     0.461824           14111.7 FPS   
SFFT                         3129       4     0.258642           7.80 Gflops   
SGEMM                        1614       4     1.127081           34.1 Gflops   
SQLite                       3302       4     0.970443        91.5 Krows/sec   
Speech Recognition           3545       4     1.689541        30.3 Words/sec   

                       Single-Core                                              
                             score threads runtime_mean            rate_string  
AES                            777       1     0.222496           599.4 MB/sec  
Camera                        2569       1     0.142803        7.12 images/sec  
Canny                         2151       1     0.171469       29.8 Mpixels/sec  
Dijkstra                      2095       1     0.374321           1.42 MTE/sec  
Face Detection                2080       1     0.124652  607.7 Ksubwindows/sec  
Gaussian Blur                 2312       1     0.126594       40.5 Mpixels/sec  
HDR                           2808       1     0.430568       10.2 Mpixels/sec  
HTML5 DOM                      292       1     2.011599    264.8 KElements/sec  
HTML5 Parse                   1578       1     0.170339            7.16 MB/sec  
Histogram Equalization        1477       1     0.249777       46.2 Mpixels/sec  
JPEG                          2485       1     0.265030       20.0 Mpixels/sec  
LLVM                          1316       1     1.486280     90.5 functions/sec  
LZMA                          1450       1     0.414550            2.27 MB/sec  
Lua                           1245       1     0.248950            1.28 MB/sec  
Memory Bandwidth              2371       1     0.099701            12.7 GB/sec  
Memory Copy                   2622       1     0.278124            7.27 GB/sec  
Memory Latency                2828       1     0.308020   6.53 Moperations/sec  
N-Body Physics                1360       1     0.273312        1.02 Mpairs/sec  
PDF Rendering                 2129       1     0.453098       56.6 Mpixels/sec  
Ray Tracing                   1854       1     0.334705      270.7 Kpixels/sec  
Rigid Body Physics            2139       1     0.240893             6262.8 FPS  